# **<mark>Triggers</mark>**

<mark>Triggers (trình kích hoạt tự động)</mark> trên SQL Server là các stored procedure đặc biệt được thực thi tự động để phản hồi đối tượng cơ sở dữ liệu, cơ sở dữ liệu và các sự kiện máy chủ. 

SQL Server cung cấp ba loại trình kích hoạt:

- <mark>Data manipulation language (Ngôn ngữ thao tác dữ liệu - DML) triggers</mark> được gọi tự động để phản hồi các sự kiện INSERT, UPDATE và DELETE so với các bảng.
- <mark>Data definition language (Ngôn ngữ định nghĩa dữ liệu - DDL) triggers</mark> để kích hoạt các câu lệnh CREATE, ALTER và DROP. Các trình kích hoạt DDL cũng kích hoạt để đáp ứng với một số thủ tục được lưu trữ trong hệ thống thực hiện các hoạt động giống như DDL.
- <mark>LOGON triggers</mark> phản hồi các sự kiện LOGON

## **<mark>CREATE TRIGGER</mark>**

Câu lệnh CREATE TRIGGER cho phép bạn tạo một trình kích hoạt mới được kích hoạt tự động bất cứ khi nào một sự kiện như INSERT, DELETE hoặc UPDATE xảy ra đối với một bảng.

```
CREATE TRIGGER [schema_name.]trigger_name
ON table_name
AFTER  {[INSERT],[UPDATE],[DELETE]}
[NOT FOR REPLICATION]
AS
{sql_statements}

```

Trong cú pháp này:

- schema\_name là tên của lược đồ chứa trình kích hoạt mới. Tên lược đồ là tùy chọn.
- trigger\_name là tên do người dùng xác định cho trình kích hoạt mới.
- table\_name là bảng mà trình kích hoạt áp dụng.
- Sự kiện được liệt kê trong mệnh đề AFTER. Sự kiện có thể là INSERT, UPDATE hoặc DELETE. Một trình kích hoạt duy nhất có thể kích hoạt để phản hồi một hoặc nhiều hành động đối với bảng.
- Tùy chọn NOT FOR REPLICATION hướng dẫn SQL Server không kích hoạt trình kích hoạt khi sửa đổi dữ liệu được thực hiện như một phần của quá trình sao chép.
- Sql\_statements là một hoặc nhiều Transact-SQL được sử dụng để thực hiện các hành động khi một sự kiện xảy ra.

### **<mark>Bảng “ảo” cho trình kích hoạt: INSERTED và DELETED</mark>**

SQL Server cung cấp hai bảng ảo có sẵn đặc biệt cho các trình kích hoạt được gọi là bảng INSERTED và DELETED. SQL Server sử dụng các bảng này để nắm bắt dữ liệu của hàng đã sửa đổi trước và sau khi sự kiện xảy ra.

| DML event | INSERTED table holds | DELETED table holds |
| --- | --- | --- |
| INSERT | rows to be inserted | empty |
| UPDATE | new rows modified by the update | existing rows modified by the update |
| DELETE | empty | rows to be deleted

**<mark>1) Tạo một bảng để ghi lại các thay đổi</mark>**

In [6]:
CREATE TABLE production.product_audits(
    change_id INT IDENTITY PRIMARY KEY,
    product_id INT NOT NULL,
    product_name VARCHAR(255) NOT NULL,
    brand_id INT NOT NULL,
    category_id INT NOT NULL,
    model_year SMALLINT NOT NULL,
    list_price DEC(10,2) NOT NULL,
    updated_at DATETIME NOT NULL,
    operation CHAR(7) NOT NULL,
    CHECK(operation IN ('INS', 'UPD-NEW', 'UPD-DEL', 'DEL'))
)

Commands completed successfully.

Total execution time: 00:00:00.006

**<mark>2) Tạo trình kích hoạt sau DML</mark>**

Đầu tiên, để tạo trình kích hoạt mới, bạn chỉ định tên của trình kích hoạt và lược đồ mà trình kích hoạt thuộc về trong mệnh đề CREATE TRIGGER:
```
CREATE TRIGGER production.trg_product_audit

```

Tiếp theo, bạn chỉ định tên của bảng, mà trình kích hoạt sẽ kích hoạt khi một sự kiện xảy ra, trong mệnh đề ON:  

```
ON production.products

```

Sau đó, bạn liệt kê một hoặc nhiều sự kiện sẽ gọi trình kích hoạt trong mệnh đề AFTER:  

```
AFTER INSERT, DELETE

```

Nội dung của trình kích hoạt bắt đầu bằng từ khóa AS:  

```
AS
BEGIN

```

Sau đó, bên trong phần thân của trình kích hoạt, bạn đặt SET NOCOUNT thành ON để ngăn chặn số lượng hàng bị ảnh hưởng gửi thông báo bất cứ khi nào trình kích hoạt được kích hoạt.  

```
SET NOCOUNT ON;

```

Trình kích hoạt sẽ chèn một hàng vào bảng production.product\_audits bất cứ khi nào một hàng được chèn vào hoặc xóa khỏi bảng production.products. Dữ liệu để chèn được cung cấp từ bảng INSERTED và DELETED thông qua toán tử UNION ALL:  

```
INSERT INTO
    production.product_audits
        (
            product_id,
            product_name,
            brand_id,
            category_id,
            model_year,
            list_price,
            updated_at,
            operation
        )
SELECT
    i.product_id,
    product_name,
    brand_id,
    category_id,
    model_year,
    i.list_price,
    GETDATE(),
    'INS'
FROM
    inserted AS i
UNION ALL
    SELECT
        d.product_id,
        product_name,
        brand_id,
        category_id,
        model_year,
        d.list_price,
        getdate(),
        'DEL'
    FROM
        deleted AS d;

```

In [2]:
CREATE TRIGGER [production].[trg_product_audit]
ON [production].[products]
AFTER INSERT, DELETE
AS
BEGIN
    SET NOCOUNT ON;

    INSERT INTO [production].[product_audits](
        [product_id], 
        [product_name],
        [brand_id],
        [category_id],
        [model_year],
        [list_price], 
        [updated_at], 
        [operation]
    )
    SELECT
        [product_id],
        [product_name],
        [brand_id],
        [category_id],
        [model_year],
        [list_price],
        GETDATE(),
        'INS'
    FROM inserted
    
    UNION ALL

    SELECT
        [product_id],
        [product_name],
        [brand_id],
        [category_id],
        [model_year],
        [list_price],
        GETDATE(),
        'DEL'
    FROM deleted
END

Commands completed successfully.

Total execution time: 00:00:00.019

In [7]:
CREATE TRIGGER [production].[trg_product_audit2]
ON [production].[products]
AFTER UPDATE
AS
BEGIN
    SET NOCOUNT ON;

    INSERT INTO [production].[product_audits](
        [product_id], 
        [product_name],
        [brand_id],
        [category_id],
        [model_year],
        [list_price], 
        [updated_at], 
        [operation]
    )
    SELECT
        [product_id],
        [product_name],
        [brand_id],
        [category_id],
        [model_year],
        [list_price],
        GETDATE(),
        'UPD-NEW'
    FROM inserted
    
    UNION ALL

    SELECT
        [product_id],
        [product_name],
        [brand_id],
        [category_id],
        [model_year],
        [list_price],
        GETDATE(),
        'UPD-DEL'
    FROM deleted
END

Commands completed successfully.

Total execution time: 00:00:00.068

**<mark>3) Kiểm tra trình kích hoạt</mark>**

In [8]:
INSERT INTO [production].[products](
    [product_name], 
    [brand_id], 
    [category_id], 
    [model_year], 
    [list_price]
)
VALUES (
    'Test product',
    1,
    1,
    2018,
    599
)

(1 row affected)

Total execution time: 00:00:00.008

Do sự kiện INSERT, trình kích hoạt production.trg\_product\_audit của bảng production.products đã được kích hoạt.

In [9]:
SELECT 
    [change_id]
    ,[product_id]
    ,[product_name]
    ,[brand_id]
    ,[category_id]
    ,[model_year]
    ,[list_price]
    ,[updated_at]
    ,[operation]
FROM 
    [production].[product_audits]

(1 row affected)

Total execution time: 00:00:00.013

change_id,product_id,product_name,brand_id,category_id,model_year,list_price,updated_at,operation
1,323,Test product,1,1,2018,599.00,2022-07-23 21:12:21.430,INS


In [10]:
UPDATE [production].[products]
SET [product_name] = 'Test Product 2'
WHERE [product_name] = 'Test product'

(1 row affected)

Total execution time: 00:00:00.013

In [11]:
SELECT 
    [change_id]
    ,[product_id]
    ,[product_name]
    ,[brand_id]
    ,[category_id]
    ,[model_year]
    ,[list_price]
    ,[updated_at]
    ,[operation]
FROM 
    [production].[product_audits]

(3 rows affected)

Total execution time: 00:00:00.009

change_id,product_id,product_name,brand_id,category_id,model_year,list_price,updated_at,operation
1,323,Test product,1,1,2018,599.00,2022-07-23 21:12:21.430,INS
2,323,Test Product 2,1,1,2018,599.00,2022-07-23 21:12:32.620,UPD-NEW
3,323,Test product,1,1,2018,599.00,2022-07-23 21:12:32.620,UPD-DEL


In [12]:
DELETE FROM 
    [production].[products]
WHERE 
    [product_name] = 'Test Product 2';


(1 row affected)

Total execution time: 00:00:00.006

In [13]:
SELECT 
    [change_id]
    ,[product_id]
    ,[product_name]
    ,[brand_id]
    ,[category_id]
    ,[model_year]
    ,[list_price]
    ,[updated_at]
    ,[operation]
FROM 
    [production].[product_audits]

(4 rows affected)

Total execution time: 00:00:00.012

change_id,product_id,product_name,brand_id,category_id,model_year,list_price,updated_at,operation
1,323,Test product,1,1,2018,599.00,2022-07-23 21:12:21.430,INS
2,323,Test Product 2,1,1,2018,599.00,2022-07-23 21:12:32.620,UPD-NEW
3,323,Test product,1,1,2018,599.00,2022-07-23 21:12:32.620,UPD-DEL
4,323,Test Product 2,1,1,2018,599.00,2022-07-23 21:13:14.977,DEL


## **<mark>DISABLE TRIGGER</mark>**

Đôi khi, vì mục đích khắc phục sự cố hoặc khôi phục dữ liệu, bạn có thể muốn tắt tạm thời một trình kích hoạt. Để thực hiện việc này, bạn sử dụng câu lệnh DISABLE TRIGGER:

```
DISABLE TRIGGER [schema_name.][trigger_name] 
ON [object_name | DATABASE | ALL SERVER]

```

Trong cú pháp này:

- Đầu tiên, chỉ định tên của lược đồ chứa trình kích hoạt và tên của trình kích hoạt mà bạn muốn vô hiệu hóa sau các từ khóa DISABLE TRIGGER.
- Thứ hai, chỉ định tên bảng hoặc dạng xem mà trình kích hoạt được liên kết với nếu trình kích hoạt là trình kích hoạt DML. Sử dụng DATABASE nếu trình kích hoạt là trình kích hoạt có phạm vi cơ sở dữ liệu DDL hoặc ALL SERVER nếu trình kích hoạt là trình kích hoạt có phạm vi máy chủ DDL.

In [ ]:
CREATE TABLE [sales].[members] (
    [member_id] INT IDENTITY PRIMARY KEY,
    [customer_id] INT NOT NULL,
    [member_level] CHAR(10) NOT NULL
)

In [ ]:
CREATE TRIGGER [sales].[trg_members_insert]
ON [sales].[members]
AFTER INSERT
AS
BEGIN
    PRINT 'A new member has been inserted'
END;

In [ ]:
INSERT INTO [sales].[members]([customer_id], [member_level])
VALUES(1,'Silver')

In [ ]:
DISABLE TRIGGER [sales].[trg_members_insert]
ON [sales].[members]

In [ ]:
INSERT INTO [sales].[members]([customer_id], [member_level])
VALUES(2,'Gold')

**<mark>Tắt tất cả trình kích hoạt trên bảng</mark>**

Để tắt tất cả các trình kích hoạt trên bảng, bạn sử dụng câu lệnh sau:

```
DISABLE TRIGGER ALL ON table_name;

```

Trong câu lệnh này, bạn chỉ cần chỉ định tên của bảng để vô hiệu hóa tất cả các trình kích hoạt thuộc về bảng đó.

In [ ]:
CREATE TRIGGER [sales].[trg_members_delete]
ON [sales].[members]
AFTER DELETE
AS
BEGIN
    PRINT 'A new member has been deleted';
END

In [ ]:
DISABLE TRIGGER ALL ON [sales].[members]

**<mark>Tắt tất cả các trình kích hoạt trên cơ sở dữ liệu</mark>**

Để tắt tất cả các trình kích hoạt trên cơ sở dữ liệu hiện tại, bạn sử dụng câu lệnh sau:
```
DISABLE TRIGGER ALL ON DATABASE;
```

## **<mark>ENABLE TRIGGER</mark>**

Câu lệnh <mark>ENABLE TRIGGER</mark> cho phép bạn bật một trình kích hoạt để trình kích hoạt có thể được kích hoạt bất cứ khi nào một sự kiện xảy ra.

Sau đây minh họa cú pháp của câu lệnh ENABLE TRIGGER:

```
ENABLE TRIGGER [schema_name.][trigger_name] 
ON [object_name | DATABASE | ALL SERVER]

```

Trong cú pháp này:

- Đầu tiên, chỉ định tên của trình kích hoạt mà bạn muốn kích hoạt. Theo tùy chọn, bạn có thể chỉ định tên của lược đồ chứa trình kích hoạt.
- Thứ hai, chỉ định bảng chứa trình kích hoạt nếu trình kích hoạt là trình kích hoạt DML. Sử dụng DATABASE nếu trình kích hoạt là trình kích hoạt có phạm vi cơ sở dữ liệu DDL hoặc ALL SERVER nếu trình kích hoạt là trình kích hoạt có phạm vi máy chủ DDL.

In [ ]:
ENABLE TRIGGER [sales].[trg_members_insert]
ON [sales].[members]

**<mark>Bật tất cả các trình kích hoạt của một bảng</mark>**

Để bật tất cả các trình kích hoạt của một bảng, bạn sử dụng câu lệnh sau:

```
ENABLE TRIGGER ALL ON table_name;

```

In [ ]:
ENABLE TRIGGER ALL ON [sales].[members]

**<mark>Bật tất cả các trình kích hoạt của cơ sở dữ liệu</mark>**

Để bật tất cả các trình kích hoạt trên cơ sở dữ liệu hiện tại, bạn sử dụng câu lệnh sau:
```
ENABLE TRIGGER ALL ON DATABASE; 

```

## **<mark>Xem định nghĩa về trình kích hoạt</mark>**

**<mark>Nhận định nghĩa trình kích hoạt bằng cách truy vấn từ chế độ xem hệ thống</mark>**

Trong truy vấn này, bạn chuyển tên của trình kích hoạt mà bạn muốn lấy định nghĩa cho hàm <mark>OBJECT\_ID ()</mark> trong mệnh đề WHERE.

In [10]:
SELECT 
    [definition]  
FROM 
    [sys].[sql_modules]  
WHERE 
    [object_id] = OBJECT_ID('sales.trg_members_delete')

(0 rows affected)

Total execution time: 00:00:00.003

definition


**<mark>Nhận định nghĩa trình kích hoạt bằng cách sử dụng hàm OBJECT\_DEFINITION</mark>**

Trong truy vấn này, bạn chuyển tên trình kích hoạt cho hàm <mark>OBJECT\_ID</mark> để lấy <mark>ID của trình kích hoạt</mark>. Sau đó, bạn sử dụng hàm <mark>OBJECT\_DEFINITION ()</mark> để lấy văn bản nguồn <mark>Transact-SQL</mark> về định nghĩa của một trình kích hoạt dựa trên <mark>ID</mark> của nó.

In [11]:
SELECT 
    OBJECT_DEFINITION (
        OBJECT_ID(
            'production.trg_product_audit'
        )
    ) AS trigger_definition

(1 row affected)

Total execution time: 00:00:00.008

trigger_definition
"CREATE TRIGGER production.trg_product_audit ON production.products AFTER INSERT, UPDATE, DELETE AS BEGIN SET NOCOUNT ON; INSERT INTO production.product_audits( product_id, product_name, brand_id, category_id, model_year, list_price, updated_at, operation ) SELECT i.product_id, product_name, brand_id, category_id, model_year, i.list_price, GETDATE(), 'INS' FROM inserted i UNION ALL SELECT i.product_id, product_name, brand_id, category_id, model_year, i.list_price, GETDATE(), 'UPD' FROM inserted i UNION ALL SELECT d.product_id, product_name, brand_id, category_id, model_year, d.list_price, GETDATE(), 'DEL' FROM deleted d; END"


**<mark>Nhận định nghĩa trình kích hoạt bằng cách sử dụng quy trình lưu trữ sp\_helptext</mark>**

In [ ]:
EXEC sp_helptext 'sales.trg_members_delete'

## **<mark>Liệt kê tất cả các trình kích hoạt</mark>**

In [ ]:
SELECT  
    [name],
    [is_instead_of_trigger]
FROM 
    [sys].[triggers]
WHERE 
    [type] = 'TR'

## **<mark>DROP TRIGGER</mark>**

Câu lệnh SQL Server DROP TRIGGER loại bỏ một hoặc nhiều trình kích hoạt khỏi cơ sở dữ liệu. Phần sau minh họa cú pháp của câu lệnh DROP TRIGGER loại bỏ các trình kích hoạt DML:

```
DROP TRIGGER [ IF EXISTS ] [schema_name.]trigger_name [ ,...n ];

```

Trong cú pháp này:

- IF EXISTS chỉ loại bỏ trình kích hoạt có điều kiện khi nó đã tồn tại.
- schema\_name là tên của lược đồ chứa trình kích hoạt DML.
- trigger\_name là tên của trình kích hoạt mà bạn muốn loại bỏ.

In [ ]:
DROP TRIGGER IF EXISTS [sales].[trg_member_insert]